In [1]:
import os
import pandas as pd
import numpy as np


## ALL SCHOOOLS (based on old mapper file)

In [2]:
allschool_locations = pd.read_csv('sg_zipcode_mapper_utf.csv', index_col = 0)
allschool_locations = allschool_locations[allschool_locations['searchval'].str.contains('SCHOOL|SKOOL')]
allschool_locations.drop(columns=['searchval','block','street_name','address','postal'], inplace=True)
allschool_locations = allschool_locations.rename(columns={'postal.1':'postal'}).reset_index(drop = True)
allschool_locations

,lat,lng,building,postal
0,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
1,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
2,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,737916
3,1.331181,103.941853,ANGLICAN HIGH SCHOOL,487012
4,1.303704,103.780363,ANGLO-CHINESE SCHOOL (INDEPENDENT),139650
...,...,...,...,...
96,1.432322,103.838094,YISHUN TOWN SECONDARY SCHOOL,768610
97,1.284212,103.825952,ZHANGDE PRIMARY SCHOOL,169485
98,1.379549,103.769313,ZHENGHUA PRIMARY SCHOOL,679002
99,1.388369,103.765522,ZHENGHUA SECONDARY SCHOOL,677741


## MOE SCHOOLS

In [3]:
moeschools = pd.read_csv('Generalinformationofmoeschools.csv')
moeschools.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


In [4]:
moeschools2 = moeschools[['school_name','address','postal_code','mrt_desc','bus_desc']]
moeschools2 = moeschools2.rename(columns={'postal_code':'postal', 'school_name':'building'})
moeschools2

,building,address,postal,mrt_desc,bus_desc
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,Admiralty Station,"TIBS 965, 964, 913"
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,ADMIRALTY MRT,904
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,Yishun,Yishun Ring Road - 812 (white plate); Yishun A...
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,"CANBERRA MRT, YISHUN MRT","117, 167, 169, 800, 811, 812, 883, 856, 858, 8..."
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,Bishan MRT,"410, 162, 52"
...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,51 Jalan Membina,169485,Tiong Bahru MRT Station,"16,33,63,123,195,605,851,14,65,147,167,196,197..."
342,ZHENGHUA PRIMARY SCHOOL,9 Fajar Road,679002,LRT: Fajar Station,"190, 184, 187, 700, 700A, 921"
343,ZHENGHUA SECONDARY SCHOOL,91 SENJA ROAD,677741,JELAPANG LRT,"920, 922, 976"
344,ZHONGHUA PRIMARY SCHOOL,12 SERANGOON AVENUE 4,556095,"Ang Mo Kio MRT, Bishan MRT, Serangoon MRT","Boundary Road: 22,24,53,105,156,315,853,501,50..."


In [5]:
geocodes = pd.read_csv('cleaned_sg_zipcode_mapper_utf.csv', index_col = 0)
geocodes = geocodes[['postal','lat','lng']]
geocodes

,postal,lat,lng
0,398614,1.312763,103.883519
1,398721,1.312390,103.881504
2,629875,1.309135,103.679463
3,439731,1.305466,103.895674
4,659592,1.344619,103.749789
...,...,...,...
26098,529393,1.351292,103.950462
26099,658712,1.342339,103.760033
26100,538884,1.357136,103.890195
26101,556095,1.360261,103.869713


In [6]:
geocodes['postal'] = [int(item) if item.isdigit() else None for item in geocodes['postal']]

In [7]:
print(type(geocodes['postal'][0]))
print(type(moeschools2['postal'][0]))

<class 'numpy.float64'>
<class 'numpy.int64'>


In [8]:
moeschools_locations = moeschools2.merge(geocodes, how = 'left', on=['postal'])
moeschools_locations

,building,address,postal,mrt_desc,bus_desc,lat,lng
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,Admiralty Station,"TIBS 965, 964, 913",1.442550,103.800214
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,ADMIRALTY MRT,904,1.445891,103.802398
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,Yishun,Yishun Ring Road - 812 (white plate); Yishun A...,1.433414,103.832751
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,"CANBERRA MRT, YISHUN MRT","117, 167, 169, 800, 811, 812, 883, 856, 858, 8...",1.436060,103.829719
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,Bishan MRT,"410, 162, 52",1.360583,103.833020
...,...,...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,51 Jalan Membina,169485,Tiong Bahru MRT Station,"16,33,63,123,195,605,851,14,65,147,167,196,197...",1.284212,103.825952
342,ZHENGHUA PRIMARY SCHOOL,9 Fajar Road,679002,LRT: Fajar Station,"190, 184, 187, 700, 700A, 921",1.379549,103.769313
343,ZHENGHUA SECONDARY SCHOOL,91 SENJA ROAD,677741,JELAPANG LRT,"920, 922, 976",1.388369,103.765522
344,ZHONGHUA PRIMARY SCHOOL,12 SERANGOON AVENUE 4,556095,"Ang Mo Kio MRT, Bishan MRT, Serangoon MRT","Boundary Road: 22,24,53,105,156,315,853,501,50...",1.360261,103.869713


In [9]:
missingschoolgeocode = moeschools_locations[moeschools_locations['lat'].isna()]
missingschoolgeocode
#missingschoolgeocode.to_excel('missingschoolgeocode.xlsx', index = False)

,building,address,postal,mrt_desc,bus_desc,lat,lng


In [10]:
moeschools_locations.to_csv('moeschools_locations.csv')

## NON MOE / PRIVATE / INTERNATIONAL SCHOOLS

In [11]:
## Locations for Higher Education Institutions (HEIs) and Private schools
## Exclude government lower education schools and special education schools
nonMOE_locations = allschool_locations[~allschool_locations['building'].str.contains("SECONDARY SCHOOL|SKOOL|PRIMARY SCHOOL|AWWA SCHOOL|RAINBOW|SAINT PATRICK'S SCHOOL|AUTISM RESOURCE CENTRE|VICTORIA SCHOOL")]
## Exclude MOE schools that does not contain the filter words
nonMOE_locations = nonMOE_locations[~nonMOE_locations['postal'].isin(moeschools['postal_code'])] #|METHODIST GIRLS' SCHOOL|CANOSSIAN SCHOOL
nonMOE_locations = nonMOE_locations.reset_index(drop = True)
nonMOE_locations


,lat,lng,building,postal
0,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
1,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
2,1.307747,103.798541,ANGLO-CHINESE SCHOOL (INTERNATIONAL),278475
3,1.311591,103.848500,APS SWIM SCHOOL,218253
4,1.350034,103.859112,AUSTRALIAN INTERNATIONAL SCHOOL,556818
5,1.345093,103.727429,CANADIAN INTERNATIONAL SCHOOL (LAKESIDE CAMPUS),649414
6,1.344857,103.792082,CHATSWORTH INTERNATIONAL SCHOOL - BUKIT TIMAH ...,289760
7,1.305674,103.779152,DOVER COURT PREPARATORY SCHOOL,139644
8,1.281218,103.834145,DUKE-NUS GRADUATE MEDICAL SCHOOL SINGAPORE,169857
9,1.302886,103.787701,ESSEC BUSINESS SCHOOL,139408


In [12]:
### DATA INVESTIGATION
# Verify if moeschools dataset in Generalinformationofschools.csv matches with the data in sg_zipcode_mapper_utf.csv
moeschools[(moeschools['postal_code'] == 599986)]


,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
171,METHODIST GIRLS' SCHOOL (PRIMARY),http://www.mgs.sch.edu.sg,11 Blackmore Drive,599986,64694800,na,64624166,na,mgps@moe.edu.sg,King Albert Park MRT,...,GIRLS' SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
172,METHODIST GIRLS' SCHOOL (SECONDARY),http://www.mgs.moe.edu.sg,11 BLACKMORE DRIVE,599986,64694800,na,64698717,na,MGSS@MOE.EDU.SG,KING ALBERT PARK MRT,...,GIRLS' SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,Yes,Chinese,Malay,Tamil


In [13]:
allschool_locations[(allschool_locations['building'] == "METHODIST GIRLS' SCHOOL")]

,lat,lng,building,postal
50,1.332948,103.783393,METHODIST GIRLS' SCHOOL,599986


In [14]:
nonMOE_locations.to_csv('nonMOE_locations.csv')

## JOIN MOE SCHOOLS WITH ALL SCHOOLS

In [15]:
allschool_locations

,lat,lng,building,postal
0,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
1,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
2,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,737916
3,1.331181,103.941853,ANGLICAN HIGH SCHOOL,487012
4,1.303704,103.780363,ANGLO-CHINESE SCHOOL (INDEPENDENT),139650
...,...,...,...,...
96,1.432322,103.838094,YISHUN TOWN SECONDARY SCHOOL,768610
97,1.284212,103.825952,ZHANGDE PRIMARY SCHOOL,169485
98,1.379549,103.769313,ZHENGHUA PRIMARY SCHOOL,679002
99,1.388369,103.765522,ZHENGHUA SECONDARY SCHOOL,677741


In [16]:
moeschools_locations = moeschools_locations[['lat','lng','building','postal']]
moeschools_locations

,lat,lng,building,postal
0,1.442550,103.800214,ADMIRALTY PRIMARY SCHOOL,738907
1,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,737916
2,1.433414,103.832751,AHMAD IBRAHIM PRIMARY SCHOOL,768643
3,1.436060,103.829719,AHMAD IBRAHIM SECONDARY SCHOOL,768928
4,1.360583,103.833020,AI TONG SCHOOL,579646
...,...,...,...,...
341,1.284212,103.825952,ZHANGDE PRIMARY SCHOOL,169485
342,1.379549,103.769313,ZHENGHUA PRIMARY SCHOOL,679002
343,1.388369,103.765522,ZHENGHUA SECONDARY SCHOOL,677741
344,1.360261,103.869713,ZHONGHUA PRIMARY SCHOOL,556095


In [17]:

allschool_locations = pd.concat([allschool_locations, moeschools_locations], ignore_index=True)

allschool_locations

,lat,lng,building,postal
0,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
1,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
2,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,737916
3,1.331181,103.941853,ANGLICAN HIGH SCHOOL,487012
4,1.303704,103.780363,ANGLO-CHINESE SCHOOL (INDEPENDENT),139650
...,...,...,...,...
442,1.284212,103.825952,ZHANGDE PRIMARY SCHOOL,169485
443,1.379549,103.769313,ZHENGHUA PRIMARY SCHOOL,679002
444,1.388369,103.765522,ZHENGHUA SECONDARY SCHOOL,677741
445,1.360261,103.869713,ZHONGHUA PRIMARY SCHOOL,556095


In [18]:
allschool_locations = allschool_locations.drop_duplicates()
allschool_locations

,lat,lng,building,postal
0,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
1,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
2,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,737916
3,1.331181,103.941853,ANGLICAN HIGH SCHOOL,487012
4,1.303704,103.780363,ANGLO-CHINESE SCHOOL (INDEPENDENT),139650
...,...,...,...,...
438,1.347186,103.722687,YUHUA SECONDARY SCHOOL,649406
439,1.351292,103.950462,YUMIN PRIMARY SCHOOL,529393
440,1.342339,103.760033,YUSOF ISHAK SECONDARY SCHOOL,658712
441,1.357136,103.890195,YUYING SECONDARY SCHOOL,538884


In [19]:
allschool_locations = allschool_locations.drop_duplicates(subset=['postal'], keep=False)
allschool_locations = allschool_locations.reset_index(drop = True)
allschool_locations

,lat,lng,building,postal
0,1.301061,103.781310,ACS INDEPENDENT BOARDING SCHOOL,139656
1,1.319923,103.834958,ACS OLDHAM HALL BOARDING SCHOOL,309937
2,1.445891,103.802398,ADMIRALTY SECONDARY SCHOOL,737916
3,1.331181,103.941853,ANGLICAN HIGH SCHOOL,487012
4,1.303704,103.780363,ANGLO-CHINESE SCHOOL (INDEPENDENT),139650
...,...,...,...,...
368,1.347186,103.722687,YUHUA SECONDARY SCHOOL,649406
369,1.351292,103.950462,YUMIN PRIMARY SCHOOL,529393
370,1.342339,103.760033,YUSOF ISHAK SECONDARY SCHOOL,658712
371,1.357136,103.890195,YUYING SECONDARY SCHOOL,538884


In [20]:
check = allschool_locations[allschool_locations.duplicated(subset=['postal'], keep=False)]
check

,lat,lng,building,postal


In [21]:
allschool_locations.to_csv('allschool_locations.csv')